In [96]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

import random

In [97]:
!kaggle competitions download -c titanic

titanic.zip: Skipping, found more recently modified local copy (use --force to force download)


In [98]:
import zipfile

zp = zipfile.ZipFile('titanic.zip') 
df = pd.read_csv(zp.open('train.csv'))

In [99]:
df = df.drop(['PassengerId', 'Name', 'Parch', 'Ticket', 'Cabin'], axis = 1)

df.loc[df['SibSp'] >= 1, 'SibSp'] = 1
df['Embarked'] = df['Embarked'].fillna('S')

df = df.dropna(subset = ['Age'])
df['Age'] = df['Age'].astype(int)

df.head()

,Survived,Pclass,Sex,Age,SibSp,Fare,Embarked
0,0,3,male,22,1,7.2500,S
1,1,1,female,38,1,71.2833,C
2,1,3,female,26,0,7.9250,S
3,1,1,female,35,1,53.1000,S
4,0,3,male,35,0,8.0500,S


In [100]:
df = pd.concat([df, pd.get_dummies(df['Sex'], prefix='Sex')],axis=1)
df = pd.concat([df, pd.get_dummies(df['Pclass'], prefix='Pclass')],axis=1)
df = pd.concat([df, pd.get_dummies(df['Embarked'], prefix='Embarked')],axis=1)
  
df = df.drop(['Sex', 'Pclass', 'Embarked'], axis = 1)
df.head()

,Survived,Age,SibSp,Fare,Sex_female,Sex_male,Pclass_1,Pclass_2,Pclass_3,Embarked_C,Embarked_Q,Embarked_S
0,0,22,1,7.2500,0,1,0,0,1,0,0,1
1,1,38,1,71.2833,1,0,1,0,0,1,0,0
2,1,26,0,7.9250,1,0,0,0,1,0,0,1
3,1,35,1,53.1000,1,0,1,0,0,0,0,1
4,0,35,0,8.0500,0,1,0,0,1,0,0,1


In [101]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(df.drop(['Survived'], axis = 1), df['Survived'], test_size=0.2)
y_train

60     0
835    1
660    1
429    1
319    1
      ..
90     0
854    0
614    0
137    0
505    0
Name: Survived, Length: 571, dtype: int64

In [252]:
class NN:
    def __init__(self, n_hidden_units = 100, learning_rate = 0.001, activation_func = 'tanh'):
        self.learning_rate = learning_rate
        self.n_hidden_units = n_hidden_units
        
        if activation_func == 'tanh':
            self.activation_func = lambda x: np.tanh(x)
            self.activation_func_derivative = lambda x: 1 - np.tanh(x)**2
        if activation_func == 'sigm':
            self.activation_func = lambda x: 1 / (1 + np.exp(-x))
            self.activation_func_derivative = lambda x: np.exp(-x) / (1 + np.exp(-x))**2
            
            
    def fit(self, X_train, y_train):
        self.X = X_train
        self.y = y_train
        
        self.n_features = len(self.X.columns)
        self.n_classes = len(self.y.unique())
        
        self.cost = lambda x: 1/len(self.y) * ((self.y * np.log(x)) + ((1 - self.y) * np.log(1 - x))).sum().sum()
        
        #z1 = x * W1 + b1
        #a1 = act_func(z1)
        #z2 = a1 * W2 + b2
        #a2 = act_func(z2)
        #y_pred = argmax(a2)
        
        self.W1 = np.random.normal(0, 0.01, (self.n_hidden_units, self.n_features))
        self.W2 = np.random.normal(0, 0.01, (self.n_classes, self.n_hidden_units))
        self.b1 = np.zeros((self.n_hidden_units, 1))
        self.b2 = np.zeros((self.n_classes, 1))
        
        for i in range(10000):
            
            #Forward propagation
            z1 = self.W1.dot(self.X.T) + self.b1
            a1 = self.activation_func(z1)
            z2 = self.W2.dot(a1) + self.b2
            a2 = self.activation_func(z2)
            
            #Backpropagation
            print(a2)
            y = np.zeros(a2.shape)
            print(y)
            #y[self.y == 1, :] = 1
            print(y)
            dz2 = a2 #- self.y
            dW2 = a1.dot(dz2.T)
            db2 = dz2
            dz1 = self.W2.dot(dz2.T) * self.activation_func_derivative(z1)
            dW1 = dz1.dot(self.X)
            db1 = dz1
            
            self.W1 -= self.learning_rate * dW1
            print(self.W2.shape, dW2.shape)
            self.W2 -= self.learning_rate * dW2
            self.b1 -= self.learning_rate * db1
            self.b2 -= self.learning_rate * db2
            
            if i % 1000 == 0:
                print("Loss after iteration %i: %f" %(i, self.W1, self.W2, self.b1, self.b2))
            

In [253]:
clf = NN()
print('tanh', clf.activation_func(np.array([0.2, 0.5])))
print('tanh derivative', clf.activation_func_derivative(np.array([0.2, 0.5])))
print()

clf = NN(activation_func = 'sigm')
print('sigmoid', clf.activation_func(np.array([0.2, 0.5])))
print('sigmoid derivative', clf.activation_func_derivative(np.array([0.2, 0.5])))

tanh [0.19737532 0.46211716]
tanh derivative [0.96104298 0.78644773]

sigmoid [0.549834   0.62245933]
sigmoid derivative [0.24751657 0.23500371]


In [254]:
clf.fit(X_train, y_train)

[[0.50439514 0.50627322 0.50726938 ... 0.50468609 0.5055868  0.50688324]
 [0.51688581 0.51421522 0.5130524  ... 0.51636864 0.51504059 0.5151355 ]]
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


ValueError: shapes (2,100) and (571,2) not aligned: 100 (dim 1) != 571 (dim 0)

In [188]:
from sklearn.neural_network import MLPClassifier

clf2 = MLPClassifier(activation = 'logistic', solver = 'sgd', learning_rate_init = 0.033)
clf2.fit(X_train, y_train)

from sklearn.metrics import accuracy_score
accuracy_score(y_test, clf2.predict(X_test))

0.7482517482517482

In [189]:
clf2 = MLPClassifier()
clf2.fit(X_train, y_train)

from sklearn.metrics import accuracy_score
accuracy_score(y_test, clf2.predict(X_test))

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\neural_network\multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


0.7972027972027972